In [1]:
teamname = '9417-hder-unsw'
data_folder='s3://tf-trachack-data/212/'
root_folder='s3://tf-trachack-notebooks/'+teamname+'/jupyter/jovyan/'

In [2]:
# import packages
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

# to display all columns
pd.set_option('display.max_columns',None)

In [3]:
dev_folder = data_folder + 'data/dev/'
eval_folder = data_folder + 'data/eval/'

In [4]:
def dataset_processing_step1(data_folder):
    '''
    In this step, we combine all the csv files except phone_info.
    Some useless features are dropped.
    Some features are transformed.
    '''    
    
    upgrades=pd.read_csv(data_folder+"upgrades.csv")
    customer_info=pd.read_csv(data_folder+"customer_info.csv")

    # drop column plan subtype
    customer_info = customer_info.drop(['plan_subtype'],axis=1)
    upgrades_custinfo=pd.merge(upgrades,customer_info.loc[:],on='line_id',how='inner')

    # redemptions
    # merge upgraded, customer info & redemptions on id & latest date
    redemptions=pd.read_csv(data_folder+"redemptions.csv")
    redempt_u = redemptions.groupby(['line_id']).max()
    ucr = pd.merge(upgrades_custinfo,redempt_u.loc[:,],on=['line_id'],how='inner')

    # suspensions
    suspensions=pd.read_csv(data_folder+"suspensions.csv")
    sus = suspensions.groupby(['line_id']).max()
    sus['suspension_start_date'] = pd.to_datetime(sus['suspension_start_date'])
    sus['suspension_end_date'] = pd.to_datetime(sus['suspension_end_date'])
    # calculate duration
    sus['suspension_duration'] = (sus['suspension_end_date'] - sus['suspension_start_date']).dt.days
    df_new=pd.merge(ucr,sus.loc[:,],on=['line_id'],how='left')

    # network_usage
    network_usage_domestic=pd.read_csv(data_folder+"network_usage_domestic.csv")
    # group by line_id
    nud = network_usage_domestic.groupby(['line_id']).sum()
    #kb to gb transform
    nud['total_gb'] = nud['total_kb']/(10**6)
    nud = nud.drop(['total_kb'],axis=1)
    nud['hotspot_gb'] = nud['hotspot_kb']/(10**6)
    nud = nud.drop(['hotspot_kb'],axis=1)
    nud['gb_5g'] = nud['kb_5g']/(10**6)
    nud = nud.drop(['kb_5g'],axis=1)
    df_new2=pd.merge(df_new,nud.loc[:,],on=['line_id'],how='inner')

    # lrp_points
    # left join as told
    lrp_points=pd.read_csv(data_folder+"lrp_points.csv")
    df_new3=pd.merge(df_new2,lrp_points.loc[:,],on=['line_id'],how='left')

    # lrp_enrollment
    lrp_enrollment=pd.read_csv(data_folder+"lrp_enrollment.csv")
    # more rows than distinct line_ids
    # groupy by line_id
    lrp_en_u = lrp_enrollment.groupby(['line_id']).max()
    df_new4=pd.merge(df_new3,lrp_en_u.loc[:,],on=['line_id'],how='left')

    #deactivation && reactivation
    #deactivation counts
    #counts how many times a line id has deactivated and keep the latest reason and date
    deactivations=pd.read_csv(data_folder+"deactivations.csv")
    reactivations=pd.read_csv(data_folder+"reactivations.csv")
    
    testdeact=deactivations
    testdeact = testdeact.groupby('line_id').count()
    dd = deactivations.groupby(['line_id']).max()
    testdeact.reset_index(inplace=True)
    testdeact=testdeact.rename(columns={'deactivation_date':'deact_count'})
    testdeact=testdeact.drop(['deactivation_reason'], axis=1)
    dea_count = pd.merge(dd,testdeact.loc[:],on='line_id',how='inner')

    #reactivation counts same as above
    testreact=reactivations.copy()
    testreact = testreact.groupby('line_id').count()
    rr = reactivations.groupby(['line_id']).max()
    testreact.reset_index(inplace=True)
    testreact=testreact.rename(columns={'reactivation_date':'react_count'})
    testreact=testreact.drop(['reactivation_channel'], axis=1)
    rea_count = pd.merge(rr,testreact.loc[:],on='line_id',how='inner')

    ea = pd.merge(dea_count,rea_count.loc[:],on='line_id',how='left')
    df_new5=pd.merge(df_new4,ea.loc[:,],on=['line_id'],how='left')

    #drop
    df_new5 = df_new5.drop(['reactivation_channel'],axis=1 )
    
    # upgrade date is heavily connected with the predict value 'upgrade', drop it.
    # revenue type are the same as redemption type, drop it
    # status are the same as enrollment, drop
    # quantity is included in total_quantity, drop it
    # if we do not keep quantity for that day, we will not need the update_date for that day
    # 'gb_5g' contains vvery few non-zero values
    df_new5 = df_new5.drop(['revenue_type','status','quantity','update_date','gb_5g'],axis=1)

    # combine them
    df_new5['total_mms'] = df_new5['mms_in'] + df_new5['mms_out'] 
    df_new5['total_sms'] = df_new5['sms_in'] + df_new5['sms_out'] 
    df_new5['total_min_voice'] = df_new5['voice_min_in'] + df_new5['voice_min_out'] 
    df_new6 = df_new5.drop(['mms_in','mms_out','sms_in','sms_out','voice_min_in','voice_min_out'],axis = 1)

    # if a customer is not in the enrolled table, it means his enrollment status is no
    df_new6['lrp_enrolled'] = df_new6['lrp_enrolled'].fillna('N')

    # if not counted, counter = 0
    df_new6['deact_count'] = df_new6['deact_count'].fillna('0')
    df_new6['react_count'] = df_new6['react_count'].fillna('0')

    # if total_quantity is nan, means 0 quantity is used
    df_new6['total_quantity'] = df_new6['total_quantity'].fillna('0')

    # 'lrp_enrollment date' is heavily connected with 'enrolled' 
    # 'voice_count_in' is included in voice_count_total
    df_new6 = df_new6.drop(['lrp_enrollment_date', 'voice_count_in','redemption_date_x'],axis = 1)
    
    # change some features that should be numeical but now is categorical into numerical
    df_new6[['total_quantity','deact_count','react_count']] = df_new6[['total_quantity','deact_count','react_count']].astype(int)
    # one hot encoding categorical features
    category_list = ['carrier','plan_name','channel','redemption_type','deactivation_reason']
    df7 = pd.get_dummies(df_new6,prefix_sep='',prefix='', columns=category_list)
    
    # change dates to year+month
    df_10 = df7.copy()
    date_cols = ['date_observed','first_activation_date','redemption_date_y','suspension_start_date','suspension_end_date','deactivation_date','reactivation_date']
    for i in date_cols:
        df_10[i]=pd.to_datetime(df_10[i],format='%Y-%m-%d')

    # latest date value, make it == 1, and the month before it be 2, and so on
    max_date = df_10['date_observed'].max()
    # fill nan with date later than now, so it will be negative
    for i in date_cols:
        df_10[i] = [(max_date - j).days for j in df_10[i]]
        
    df11=df_10.copy()
    
    y_C = [ 'deactivation_date', 'reactivation_date','suspension_start_date', 'suspension_end_date']
    X_C = [['first_activation_date'], ['first_activation_date'], ['first_activation_date','deactivation_date','reactivation_date'],
           ['first_activation_date','deactivation_date', 'reactivation_date','suspension_start_date']]

    df12_nonan = df11.dropna()
    df12_with_nan = df11.copy()
    for i in range(0, len(y_C)):
        # use knn regressor
        knnR = KNeighborsRegressor(n_neighbors = 8)
        model_knnR = knnR.fit(df12_nonan[X_C[i]],df12_nonan[y_C[i]])

        y_with_missing = df12_with_nan[y_C[i]]
        df_missing_x = pd.DataFrame(df12_with_nan[X_C[i]])
        pred_y = model_knnR.predict(df_missing_x)

        indicies_of_missing = df12_with_nan[df12_with_nan[y_C[i]].isnull()].index # list of the missing indices
        for fill_index, dataframe_index in enumerate(indicies_of_missing):
            if pd.isnull(df12_with_nan.loc[dataframe_index, y_C[i]]):
                # fill null
                df12_with_nan.loc[dataframe_index,y_C[i]] = pred_y[fill_index]
            else:
                continue
                
    df13 = df12_with_nan.copy()
    df13['suspension_duration'] = df13['suspension_start_date'] - df13['suspension_end_date']
    df14 = df13.copy()

    #normalize dates
    date_list = ['first_activation_date','redemption_date_y','suspension_start_date','suspension_end_date','suspension_duration','deactivation_date','reactivation_date']

    for i in date_list:
        df14[i]=df14[i].astype(int)
        df14[i]=(df14[i]-df14[i].min())/(df14[i].max()-df14[i].min())


    #yes no converstion
    df14.replace({'upgrade': 'yes', 'lrp_enrolled': 'Y'}, 1, inplace=True)
    df14.replace({'upgrade': 'no', 'lrp_enrolled': 'N'}, 0, inplace=True)
    
    return df14

In [5]:
dev_step1 = dataset_processing_step1(dev_folder)
dev_step1

,line_id,date_observed,upgrade,first_activation_date,gross_revenue,redemption_date_y,suspension_start_date,suspension_end_date,suspension_duration,voice_count_total,total_gb,hotspot_gb,total_quantity,lrp_enrolled,deactivation_date,deact_count,reactivation_date,react_count,total_mms,total_sms,total_min_voice,carrier 1,carrier 2,carrier 3,Other,plan 1,plan 2,plan 3,plan 4,API,APP,BATCH,BOT,HANDSET,IVR,JACADA,REWARDS,RTR,SMS,TAS,VMBC,WARP,WEB,WEBCSR,FREE,MPPAID,PAID,Replacement,ACTIVE UPGRADE,CHANGE OF ADDRESS,CUSTOMER REQD,DEFECTIVE,DEVICE CHANGE INQUIRY,DEVICERETURN,MINCHANGE,NO NEED OF PHONE,PASTDUE,PORT CANCEL,PORT OUT,PORTED NO A/I,REFURBISHED,REMOVED_FROM_GROUP,RISK ASSESSMENT,SENDCARRDEACT,SIM EXCHANGE,STOLEN,STOLEN CREDIT CARD,UPGRADE,WN-SYSTEM ISSUED
0,f0b99d3b-32f7-4464-8fa8-87f27c9810a8,0,1,0.287792,50.0,0.036199,0.231394,0.232390,0.648595,40,51.691485,0.000000,0,1,0.131044,0,0.186536,0,873,837,97,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,a2ec0baf-1720-4b1b-bfa3-db00ac8fcb00,4,1,0.212663,55.0,0.022624,0.002833,0.002848,0.648452,623,94.619520,0.009536,0,1,0.010512,3,0.010519,3,75,446,1143,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,dd3a01a3-7baf-4605-9799-ce434328a20d,4,0,0.053923,35.0,0.027149,0.003400,0.003418,0.648452,298,33.641029,0.000000,0,0,0.012614,4,0.012623,4,37,1361,584,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,5cf5917c-f407-4cc7-9d6d-dafe8ec0eccc,1,0,0.013329,35.0,0.022624,0.043257,0.224796,0.512341,139,10.616986,0.000000,0,0,0.070778,0,0.067321,0,270,1390,1522,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,6437fcbd-02c4-4c52-b5b2-f7e535ecec47,4,1,0.058770,54.0,0.027149,0.026067,0.026201,0.648452,1091,184.177785,0.000000,0,0,0.096706,1,0.096774,1,183,2148,4043,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54853,f89b108a-f453-4419-a4dc-5bf8e1cd1f1a,3,0,0.052711,130.0,0.116139,0.014923,0.014619,0.648737,2260,19.400427,0.000000,0,0,0.055361,2,0.053997,2,40,2298,3571,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
54854,b703f1d4-57ba-48b6-8adf-46d7aba6a406,1,0,0.205998,55.0,0.033183,0.015489,0.015569,0.648452,1178,230.961530,0.516887,3049,1,0.057463,6,0.057504,6,1583,26266,4715,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
54855,e3b2212c-906e-4a8f-8307-6b88fc64d7a0,3,0,0.016965,45.0,0.039216,0.003589,0.003038,0.648737,276,52.458746,0.000000,0,1,0.124036,0,0.038569,0,19,1011,562,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
54856,828df9bf-279c-47c3-96c2-92177a50b8fd,0,0,0.014541,35.0,0.025641,0.003211,0.003228,0.648452,769,0.000088,0.000000,0,0,0.011913,1,0.011921,1,0,612,2877,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
eval_step1 = dataset_processing_step1(eval_folder)
eval_step1

,line_id,date_observed,first_activation_date,gross_revenue,redemption_date_y,suspension_start_date,suspension_end_date,suspension_duration,voice_count_total,total_gb,hotspot_gb,total_quantity,lrp_enrolled,deactivation_date,deact_count,reactivation_date,react_count,total_mms,total_sms,total_min_voice,carrier 1,carrier 2,carrier 3,Other,plan 1,plan 2,plan 3,plan 4,APP,BATCH,BOT,HANDSET,IVR,JACADA,REWARDS,RTR,SMS,TAS,WARP,WEB,WEBCSR,FREE,MPPAID,PAID,Replacement,ACTIVE UPGRADE,CHANGE OF ADDRESS,CUSTOMER REQD,DEVICE CHANGE INQUIRY,MINCHANGE,NO NEED OF PHONE,PASTDUE,PORT CANCEL,PORT OUT,PORTED NO A/I,REFURBISHED,REMOVED_FROM_GROUP,RISK ASSESSMENT,SENDCARRDEACT,STOLEN,STOLEN CREDIT CARD,UPGRADE,WN-SYSTEM ISSUED
0,584f0d75-5897-4e56-b475-0b3adf1aeb07,4,0.298059,44.0,0.027875,0.082311,0.082435,0.623019,1663,75.316809,0.000000,0,0,0.305322,1,0.305322,1,133,258,4046,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,e879756a-daff-4f72-9206-d96009861ac1,4,0.350344,42.5,0.029617,0.320559,0.320855,0.623167,376,104.895317,0.000000,0,0,0.161064,0,0.166667,0,311,3418,1118,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,cbb0795d-716a-44f1-bc82-f8f8cb5db002,4,0.058860,55.0,0.109756,0.017935,0.012479,0.627314,162,19.601947,4.693261,0,0,0.022409,4,0.044118,4,21,1742,721,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,e1e914fd-1e04-4148-9251-ed6a4dd6d3e7,1,0.445523,15.0,0.022648,0.291297,0.286254,0.627314,624,0.000000,0.000000,0,0,0.041317,0,0.050420,0,0,0,9667,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1987025d-39be-4120-a09a-8d89c0e8d151,0,0.030996,55.0,0.060976,0.006608,0.006618,0.623019,278,1.279978,0.000000,0,0,0.002801,3,0.024510,3,75,1798,737,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36486,bca0aa56-6582-4d01-b67a-3cf8a10c7632,1,0.006575,45.0,0.036585,0.010383,0.009265,0.623759,253,7.852177,0.000000,0,0,0.005602,0,0.032913,0,13,1718,902,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
36487,e857da2f-aaac-47d6-80f5-d475e87be7b2,1,0.004696,35.0,0.026132,0.009251,0.009075,0.623167,76,0.047078,0.000000,0,0,0.014706,0,0.021709,0,2,90,372,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
36488,ab633967-4f11-435b-a755-13015d242364,3,0.016594,35.0,0.041812,0.023787,0.018340,0.627314,49,10.892607,0.000000,0,0,0.011204,0,0.041317,0,262,169,59,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
36489,ab9dda24-b688-4cb8-9ff8-27158438ff4d,0,0.158735,130.0,0.024390,0.008495,0.008319,0.623167,468,14.797651,0.000000,0,1,0.031513,6,0.030812,6,17,684,4094,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


### Since dev_step1 has more features than eval_step1，we need to fill up those features

All of them are generated by getdummies(), so we just need to fill 0 to those features.

In [7]:
dev_columns = dev_step1.columns
eval_columns = eval_step1.columns

for e in dev_columns:
    if e not in eval_columns:
        print(e)

print('----')
for e in eval_columns:
    if e not in dev_columns:
        print(e)

upgrade
API
VMBC
DEFECTIVE
DEVICERETURN
SIM EXCHANGE
----


In [8]:
eval_step1['API'] = 0
eval_step1['VMBS'] = 0
eval_step1['DEFECTIVE'] = 0
eval_step1['DEVICERETURN'] = 0
eval_step1['SIM EXCHANGE'] = 0

In [9]:
eval_step1

,line_id,date_observed,first_activation_date,gross_revenue,redemption_date_y,suspension_start_date,suspension_end_date,suspension_duration,voice_count_total,total_gb,hotspot_gb,total_quantity,lrp_enrolled,deactivation_date,deact_count,reactivation_date,react_count,total_mms,total_sms,total_min_voice,carrier 1,carrier 2,carrier 3,Other,plan 1,plan 2,plan 3,plan 4,APP,BATCH,BOT,HANDSET,IVR,JACADA,REWARDS,RTR,SMS,TAS,WARP,WEB,WEBCSR,FREE,MPPAID,PAID,Replacement,ACTIVE UPGRADE,CHANGE OF ADDRESS,CUSTOMER REQD,DEVICE CHANGE INQUIRY,MINCHANGE,NO NEED OF PHONE,PASTDUE,PORT CANCEL,PORT OUT,PORTED NO A/I,REFURBISHED,REMOVED_FROM_GROUP,RISK ASSESSMENT,SENDCARRDEACT,STOLEN,STOLEN CREDIT CARD,UPGRADE,WN-SYSTEM ISSUED,API,VMBS,DEFECTIVE,DEVICERETURN,SIM EXCHANGE
0,584f0d75-5897-4e56-b475-0b3adf1aeb07,4,0.298059,44.0,0.027875,0.082311,0.082435,0.623019,1663,75.316809,0.000000,0,0,0.305322,1,0.305322,1,133,258,4046,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,e879756a-daff-4f72-9206-d96009861ac1,4,0.350344,42.5,0.029617,0.320559,0.320855,0.623167,376,104.895317,0.000000,0,0,0.161064,0,0.166667,0,311,3418,1118,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,cbb0795d-716a-44f1-bc82-f8f8cb5db002,4,0.058860,55.0,0.109756,0.017935,0.012479,0.627314,162,19.601947,4.693261,0,0,0.022409,4,0.044118,4,21,1742,721,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,e1e914fd-1e04-4148-9251-ed6a4dd6d3e7,1,0.445523,15.0,0.022648,0.291297,0.286254,0.627314,624,0.000000,0.000000,0,0,0.041317,0,0.050420,0,0,0,9667,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1987025d-39be-4120-a09a-8d89c0e8d151,0,0.030996,55.0,0.060976,0.006608,0.006618,0.623019,278,1.279978,0.000000,0,0,0.002801,3,0.024510,3,75,1798,737,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36486,bca0aa56-6582-4d01-b67a-3cf8a10c7632,1,0.006575,45.0,0.036585,0.010383,0.009265,0.623759,253,7.852177,0.000000,0,0,0.005602,0,0.032913,0,13,1718,902,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
36487,e857da2f-aaac-47d6-80f5-d475e87be7b2,1,0.004696,35.0,0.026132,0.009251,0.009075,0.623167,76,0.047078,0.000000,0,0,0.014706,0,0.021709,0,2,90,372,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
36488,ab633967-4f11-435b-a755-13015d242364,3,0.016594,35.0,0.041812,0.023787,0.018340,0.627314,49,10.892607,0.000000,0,0,0.011204,0,0.041317,0,262,169,59,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
36489,ab9dda24-b688-4cb8-9ff8-27158438ff4d,0,0.158735,130.0,0.024390,0.008495,0.008319,0.623167,468,14.797651,0.000000,0,1,0.031513,6,0.030812,6,17,684,4094,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
def dataset_processing_step2(data_folder, df14):
    '''
    In this step, we will process phone_info.csv
    '''
    phone_info=pd.read_csv(data_folder+"phone_info.csv")
    testphone = phone_info.set_index('line_id')
    nnull = testphone[testphone.isna().all(axis=1)]
    nnull.reset_index(inplace=True)
    testphone.reset_index(inplace=True)
    cond = testphone['line_id'].isin(nnull['line_id'])
    testphone.drop(testphone[cond].index, inplace = True)
    df15=pd.merge(df14,testphone.loc[:],on='line_id',how='left')
    
    df141 = df15.copy()
    latest_year = df141['year_released'].max()+1
    df141['year_released'] = df141['year_released'].fillna(latest_year+5)
    df141['year_released'] = [latest_year - int(i) for i in df141['year_released']]

    # year_released
    min_year = df141['year_released'].max()
    for yy in df141['year_released'].unique():
        if yy < min_year and yy > 0:
            min_year = yy
    min_year = min_year - 1        
    max_year = df141['year_released'].max()
    df14_norm = df141.copy()
    df14_norm['year_released'] = (df14_norm['year_released']-min_year)/(max_year-min_year)
    df14_norm['year_released'][df14_norm['year_released']<0]=np.nan
    
    # cpu_cores
    cpu_tran = df14_norm[['line_id','cpu_cores']].dropna()
    cpu_tran = cpu_tran.set_index('line_id')
    for i in range(0, len(cpu_tran['cpu_cores'])):
        cpu_list = list(map(int,cpu_tran['cpu_cores'][i].split('+')))
        cpu_tran['cpu_cores'][i] = sum(cpu_list)

    df15 = df14_norm.set_index('line_id')
    df15.loc[df15.index.isin(cpu_tran.index), ['cpu_cores']] = cpu_tran['cpu_cores']
    df15.reset_index(inplace=True)
    
    # deal with internal_storage_capacity
    # calculate average and /100, e.g. 128->1.28
    inter_tran = df15[['line_id','internal_storage_capacity']].dropna()
    inter_tran = inter_tran.set_index('line_id')
    for i in range(0, len(inter_tran['internal_storage_capacity'])):
        inter_list = list(map(float,inter_tran['internal_storage_capacity'][i].split('/')))
        # deal with some values
        if inter_list == [0.32]:
            inter_list = [32]
        elif inter_list == [0.125]:
            inter_list = [128]
        elif inter_list == [0.256]:
            inter_list = [256]
        inter_tran['internal_storage_capacity'][i] = (sum(inter_list)/len(inter_list))/100
    df16 = df15.set_index('line_id')
    df16.loc[df16.index.isin(inter_tran.index), ['internal_storage_capacity']] = inter_tran['internal_storage_capacity']
    df16.reset_index(inplace=True)
    
    # total ram
    ram_tran = df16[['line_id','total_ram']].dropna()
    ram_tran = ram_tran.set_index('line_id')
    for i in range(0, len(ram_tran['total_ram'])):
        ram_list = list(map(int,ram_tran['total_ram'][i].split('/')))
        ram_tran['total_ram'][i] = (sum(ram_list)/len(ram_list))/1000
    df17 = df16.set_index('line_id')
    df17.loc[df17.index.isin(ram_tran.index), ['total_ram']] = ram_tran['total_ram']
    df17.reset_index(inplace=True)
    
    #convert v1 into 1
    df17['os_version'][df17['os_version']=='V1.11.19'] = '1'

    #retain the first number
    df17['os_version'] = df17['os_version'].astype('str')
    df17['os_version'] = df17['os_version'].str.replace(r"\..*$","")
    df17['os_version'] = df17['os_version'].str.replace(r"\_.*$","")
    df17['os_version'][df17['os_version']=='nan']=np.nan
    
    # drop lines which have gsma_device_type missing
    df18=df17.drop(['os_family','os_name', 'os_vendor'],axis = 1)
    df18 = df18.set_index('line_id')
    nnull = df18[df18['gsma_device_type'].isna()]

    nnull.reset_index(inplace=True)
    df18.reset_index(inplace=True)
    cond = df18['line_id'].isin(nnull['line_id'])
    df18.drop(df18[cond].index, inplace = True)
    
    # gsma operating system
    df18['gsma_operating_system'][df18['gsma_operating_system']=='Android,Not Known']='Android'
    df18['gsma_operating_system'][df18['gsma_operating_system']=='Android,MTK']='Android'
    df18['gsma_operating_system'][df18['gsma_operating_system']=='Android,SC6800H']='Android'
    df18['gsma_operating_system'][df18['gsma_operating_system']=='Not known,iOS']='IOS'
    df18['gsma_operating_system'][df18['gsma_operating_system']=='Not known']='IOS'
    df18['gsma_operating_system'][df18['gsma_operating_system']=='Not Known,iOS']='IOS'
    df18['gsma_operating_system'][df18['gsma_operating_system']=='NONE']='KaiOS'

    df18['gsma_operating_system'][df18['gsma_operating_system']=='Not Known']=np.nan
    
    df18['gsma_model_name'] = df18['gsma_model_name'].astype('str')
    df18['gsma_model_name'] = df18['gsma_model_name'].str.replace(r"\(.*\)","")
    
    object_to_int_list = ['cpu_cores','os_version']
    object_to_float_list = ['internal_storage_capacity', 'total_ram']

    for i in object_to_int_list:
        df18[i] = df18[i].fillna(-100)
    for i in object_to_float_list: 
        df18[i] = df18[i].fillna(-100)

    df18[object_to_int_list] = df18[object_to_int_list].astype(int)
    df18[object_to_float_list] = df18[object_to_float_list].astype(float)

    for i in object_to_int_list:
        df18[i][df18[i]<0]=np.nan
    for i in object_to_float_list: 
        df18[i][df18[i]<0]=np.nan
     
    # some other features
    df19 = df18.copy()
    df19[['gsma_device_type','gsma_model_name','gsma_operating_system','gsma_device_type','manufacturer', 'sim_size']]=df19[['gsma_device_type','gsma_model_name','gsma_operating_system','gsma_device_type','manufacturer', 'sim_size']].astype('str')
    label_list = ['gsma_device_type','gsma_model_name','gsma_operating_system','gsma_device_type','manufacturer', 'sim_size']
    label_list_with_nan = ['manufacturer', 'sim_size']
    legdt=LabelEncoder()
    df19['gsma_device_type'] = legdt.fit_transform(df19['gsma_device_type'])
    legmn = LabelEncoder()
    df19['gsma_model_name'] = legmn.fit_transform(df19['gsma_model_name'])
    legos = LabelEncoder()
    df19['gsma_operating_system'] = legos.fit_transform(df19['gsma_operating_system'])
    lem = LabelEncoder()
    df19['manufacturer'] = lem.fit_transform(df19['manufacturer'])
    lss = LabelEncoder()
    df19['sim_size'] = lss.fit_transform(df19['sim_size'])

    for i in label_list_with_nan:
        df19[i][df19[i]==df19[i].max()]=np.nan
        
    df_final = df19.copy()

    tree_input = df19[['gsma_model_name','manufacturer']].dropna()
    treex = np.array(tree_input['gsma_model_name']).reshape(-1, 1)
    X_train, X_test, y_train, y_test = train_test_split(treex, tree_input['manufacturer'], test_size=0.2, random_state=1)
    tr = DecisionTreeClassifier(random_state=0)
    # Fit the classifier to the data
    model = tr.fit(X_train,y_train)

    #fill in process
    dfx_input = np.array(df_final['gsma_model_name']).reshape(-1, 1)
    pred_y = model.predict(dfx_input)

    indicies_of_missing = df_final[df_final['manufacturer'].isnull()].index # list of the missing indices
    for fill_index, dataframe_index in enumerate(indicies_of_missing):
        if pd.isnull(df_final.loc[dataframe_index, 'manufacturer']):
            df_final.loc[dataframe_index,'manufacturer'] = pred_y[fill_index]
        else:
            continue
    
    # since we need to use 'year_released' later, so do it first
    # this is the dataFrame have no missing 
    # used to build model
    df19_no_missing = df19.dropna()
    y_r = ['year_released','internal_storage_capacity']
    X_r = [['gsma_operating_system','manufacturer','gsma_model_name'],['gsma_model_name','gsma_operating_system','manufacturer']]

    for i in range(0, len(y_r)):

        knnR = KNeighborsRegressor(n_neighbors = 4)
        model_knnR = knnR.fit(df19_no_missing[X_r[i]],df19_no_missing[y_r[i]])

        y_with_missing = df_final[y_r[i]].astype(float)
        df_missing_x = pd.DataFrame(df_final[X_r[i]])
        pred_y = model_knnR.predict(df_missing_x)

        indicies_of_missing = df_final[df_final[y_r[i]].isnull()].index # list of the missing indices
        for fill_index, dataframe_index in enumerate(indicies_of_missing):
            if pd.isnull(df_final.loc[dataframe_index, y_r[i]]):
                df_final.loc[dataframe_index,y_r[i]] = pred_y[fill_index]
            else:
                continue
    
    # this is the dataFrame have no missing 
    # used to build model
    df19_no_missing = df19.dropna()
    y_c = ['lte_advanced','cpu_cores', 'expandable_storage', 'lte_category','touch_screen', 'sim_size', 'os_version']
    X_c = [['gsma_device_type'], ['gsma_operating_system','manufacturer','year_released'], ['gsma_model_name','gsma_operating_system','manufacturer'],['cpu_cores','year_released'], ['gsma_device_type'],['gsma_operating_system','manufacturer'],['gsma_model_name','gsma_operating_system','gsma_device_type']]

    for i in range(0,len(y_c)):

        treeC = DecisionTreeClassifier(random_state=0)
        model_tree = treeC.fit(df19_no_missing[X_c[i]],df19_no_missing[y_c[i]])

        y_with_missing = df_final[y_c[i]]
        df_missing_x = pd.DataFrame(df_final[X_c[i]])
        pred_y = model_tree.predict(df_missing_x)

        indicies_of_missing = df_final[df_final[y_c[i]].isnull()].index # list of the missing indices
        for fill_index, dataframe_index in enumerate(indicies_of_missing):
            if pd.isnull(df_final.loc[dataframe_index, y_c[i]]):
                df_final.loc[dataframe_index,y_c[i]] = pred_y[fill_index]
            else:
                continue
                
    # since we need 'cpu_cores' and 'lte_category' to fill 'total_ram'
    # so we do this now
    df19_no_missing = df19.dropna()
    y_r = ['total_ram']
    X_r = [['cpu_cores','lte_category','internal_storage_capacity', 'year_released','os_version']]

    for i in range(0, len(y_r)):

        knnR = KNeighborsRegressor(n_neighbors = 4)
        model_knnR = knnR.fit(df19_no_missing[X_r[i]],df19_no_missing[y_r[i]])

        y_with_missing = df_final[y_r[i]]
        df_missing_x = pd.DataFrame(df_final[X_r[i]])
        pred_y = model_knnR.predict(df_missing_x)

        indicies_of_missing = df_final[df_final[y_r[i]].isnull()].index # list of the missing indices
        for fill_index, dataframe_index in enumerate(indicies_of_missing):
            if pd.isnull(df_final.loc[dataframe_index, y_r[i]]):
                df_final.loc[dataframe_index,y_r[i]] = pred_y[fill_index]
            else:
                continue
    
    # this is the dataFrame have no missing 
    # used to build model
    df19_no_missing = df19.dropna()
    y_c = ['lte','wi_fi']
    X_c = [['gsma_device_type','HANDSET','lte_advanced','touch_screen'],['gsma_device_type','HANDSET','lte_advanced','touch_screen','lte']]

    for i in range(0,len(y_c)):

        treeC = DecisionTreeClassifier(random_state=0)
        model_tree = treeC.fit(df19_no_missing[X_c[i]],df19_no_missing[y_c[i]])

        y_with_missing = df_final[y_c[i]]
        df_missing_x = pd.DataFrame(df_final[X_c[i]])
        pred_y = model_tree.predict(df_missing_x)

        indicies_of_missing = df_final[df_final[y_c[i]].isnull()].index # list of the missing indices
        for fill_index, dataframe_index in enumerate(indicies_of_missing):
            if pd.isnull(df_final.loc[dataframe_index, y_c[i]]):
                df_final.loc[dataframe_index,y_c[i]] = pred_y[fill_index]
            else:
                continue
                
    return df_final

In [11]:
dev_final = dataset_processing_step2(dev_folder, dev_step1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:86: SettingWithCopyWarning: 
A value is trying to b

In [12]:
dev_final

,line_id,date_observed,upgrade,first_activation_date,gross_revenue,redemption_date_y,suspension_start_date,suspension_end_date,suspension_duration,voice_count_total,total_gb,hotspot_gb,total_quantity,lrp_enrolled,deactivation_date,deact_count,reactivation_date,react_count,total_mms,total_sms,total_min_voice,carrier 1,carrier 2,carrier 3,Other,plan 1,plan 2,plan 3,plan 4,API,APP,BATCH,BOT,HANDSET,IVR,JACADA,REWARDS,RTR,SMS,TAS,VMBC,WARP,WEB,WEBCSR,FREE,MPPAID,PAID,Replacement,ACTIVE UPGRADE,CHANGE OF ADDRESS,CUSTOMER REQD,DEFECTIVE,DEVICE CHANGE INQUIRY,DEVICERETURN,MINCHANGE,NO NEED OF PHONE,PASTDUE,PORT CANCEL,PORT OUT,PORTED NO A/I,REFURBISHED,REMOVED_FROM_GROUP,RISK ASSESSMENT,SENDCARRDEACT,SIM EXCHANGE,STOLEN,STOLEN CREDIT CARD,UPGRADE,WN-SYSTEM ISSUED,cpu_cores,expandable_storage,gsma_device_type,gsma_model_name,gsma_operating_system,internal_storage_capacity,lte,lte_advanced,lte_category,manufacturer,os_version,sim_size,total_ram,touch_screen,wi_fi,year_released
0,f0b99d3b-32f7-4464-8fa8-87f27c9810a8,0,1,0.287792,50.0,0.036199,0.231394,0.232390,0.648595,40,51.691485,0.000000,0,1,0.131044,0,0.186536,0,873,837,97,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.0,1.0,3,602,0,0.160000,1.0,1.0,4.0,28.0,6.0,4.0,1.536,1.0,1.0,0.30
1,a2ec0baf-1720-4b1b-bfa3-db00ac8fcb00,4,1,0.212663,55.0,0.022624,0.002833,0.002848,0.648452,623,94.619520,0.009536,0,1,0.010512,3,0.010519,3,75,446,1143,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,6.0,0.0,3,815,7,1.493333,1.0,1.0,12.0,1.0,12.0,4.0,3.072,1.0,1.0,0.20
2,dd3a01a3-7baf-4605-9799-ce434328a20d,4,0,0.053923,35.0,0.027149,0.003400,0.003418,0.648452,298,33.641029,0.000000,0,0,0.012614,4,0.012623,4,37,1361,584,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,4.0,0.0,3,809,7,1.386667,1.0,1.0,9.0,1.0,10.0,4.0,2.048,1.0,1.0,0.40
3,5cf5917c-f407-4cc7-9d6d-dafe8ec0eccc,1,0,0.013329,35.0,0.022624,0.043257,0.224796,0.512341,139,10.616986,0.000000,0,0,0.070778,0,0.067321,0,270,1390,1522,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.0,0.0,3,809,7,1.386667,1.0,1.0,9.0,1.0,10.0,4.0,2.048,1.0,1.0,0.40
4,6437fcbd-02c4-4c52-b5b2-f7e535ecec47,4,1,0.058770,54.0,0.027149,0.026067,0.026201,0.648452,1091,184.177785,0.000000,0,0,0.096706,1,0.096774,1,183,2148,4043,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,8.0,1.0,3,493,0,0.640000,1.0,1.0,18.0,28.0,8.0,4.0,6.144,1.0,1.0,0.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54852,fbd17822-08f2-4863-aa91-9891ffbe5002,3,1,0.070585,45.0,0.033183,0.004156,0.004177,0.648452,864,31.842505,0.000000,0,1,0.015417,5,0.015428,5,199,3351,2830,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,4.0,1.0,3,737,0,1.600000,1.0,1.0,6.0,28.0,9.0,4.0,2.048,1.0,1.0,0.45
54853,f89b108a-f453-4419-a4dc-5bf8e1cd1f1a,3,0,0.052711,130.0,0.116139,0.014923,0.014619,0.648737,2260,19.400427,0.000000,0,0,0.055361,2,0.053997,2,40,2298,3571,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,4.0,1.0,3,598,0,1.386667,1.0,1.0,4.0,28.0,9.0,4.0,2.048,1.0,1.0,0.40
54854,b703f1d4-57ba-48b6-8adf-46d7aba6a406,1,0,0.205998,55.0,0.033183,0.015489,0.015569,0.648452,1178,230.961530,0.516887,3049,1,0.057463,6,0.057504,6,1583,26266,4715,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,4.0,0.0,3,810,7,1.386667,1.0,1.0,9.0,1.0,10.0,4.0,3.072,1.0,1.0,0.40
54856,828df9bf-279c-47c3-96c2-92177a50b8fd,0,0,0.014541,35.0,0.025641,0.003211,0.003228,0

In [13]:
eval_final = dataset_processing_step2(eval_folder, eval_step1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:86: SettingWithCopyWarning: 
A value is trying to b

In [14]:
eval_final

,line_id,date_observed,first_activation_date,gross_revenue,redemption_date_y,suspension_start_date,suspension_end_date,suspension_duration,voice_count_total,total_gb,hotspot_gb,total_quantity,lrp_enrolled,deactivation_date,deact_count,reactivation_date,react_count,total_mms,total_sms,total_min_voice,carrier 1,carrier 2,carrier 3,Other,plan 1,plan 2,plan 3,plan 4,APP,BATCH,BOT,HANDSET,IVR,JACADA,REWARDS,RTR,SMS,TAS,WARP,WEB,WEBCSR,FREE,MPPAID,PAID,Replacement,ACTIVE UPGRADE,CHANGE OF ADDRESS,CUSTOMER REQD,DEVICE CHANGE INQUIRY,MINCHANGE,NO NEED OF PHONE,PASTDUE,PORT CANCEL,PORT OUT,PORTED NO A/I,REFURBISHED,REMOVED_FROM_GROUP,RISK ASSESSMENT,SENDCARRDEACT,STOLEN,STOLEN CREDIT CARD,UPGRADE,WN-SYSTEM ISSUED,API,VMBS,DEFECTIVE,DEVICERETURN,SIM EXCHANGE,cpu_cores,expandable_storage,gsma_device_type,gsma_model_name,gsma_operating_system,internal_storage_capacity,lte,lte_advanced,lte_category,manufacturer,os_version,sim_size,total_ram,touch_screen,wi_fi,year_released
0,584f0d75-5897-4e56-b475-0b3adf1aeb07,4,0.298059,44.0,0.027875,0.082311,0.082435,0.623019,1663,75.316809,0.000000,0,0,0.305322,1,0.305322,1,133,258,4046,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.0,0.0,2,705,7,1.386667,1.0,1.0,9.0,1.0,10.0,3.0,2.048,1.0,1.0,0.400
1,e879756a-daff-4f72-9206-d96009861ac1,4,0.350344,42.5,0.029617,0.320559,0.320855,0.623167,376,104.895317,0.000000,0,0,0.161064,0,0.166667,0,311,3418,1118,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.0,0.0,2,710,7,1.600000,1.0,1.0,12.0,1.0,11.0,3.0,3.072,1.0,1.0,0.300
2,cbb0795d-716a-44f1-bc82-f8f8cb5db002,4,0.058860,55.0,0.109756,0.017935,0.012479,0.627314,162,19.601947,4.693261,0,0,0.022409,4,0.044118,4,21,1742,721,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,0.0,2,704,7,0.600000,1.0,1.0,6.0,1.0,9.0,3.0,2.048,1.0,1.0,0.500
4,1987025d-39be-4120-a09a-8d89c0e8d151,0,0.030996,55.0,0.060976,0.006608,0.006618,0.623019,278,1.279978,0.000000,0,0,0.002801,3,0.024510,3,75,1798,737,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.0,1.0,2,534,0,0.160000,1.0,1.0,4.0,28.0,6.0,3.0,2.048,1.0,1.0,0.300
6,fa8fe72b-6647-4f54-92ba-f3ea71b6905f,4,0.385097,55.0,0.045296,0.006419,0.004916,0.624204,220,68.067106,0.000000,6419,1,0.023810,6,0.018207,6,173,1175,629,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,0.0,2,701,7,0.600000,1.0,1.0,4.0,1.0,8.0,3.0,1.024,1.0,1.0,0.600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36485,ff45cd33-9d52-4766-bf09-2ece765af186,0,0.340326,45.0,0.045296,0.006608,0.004916,0.624352,146,22.496349,0.000000,0,0,0.024510,23,0.018207,24,4,167,598,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.0,0.0,2,708,7,0.960000,1.0,1.0,12.0,1.0,11.0,3.0,3.072,1.0,1.0,0.300
36486,bca0aa56-6582-4d01-b67a-3cf8a10c7632,1,0.006575,45.0,0.036585,0.010383,0.009265,0.623759,253,7.852177,0.000000,0,0,0.005602,0,0.032913,0,13,1718,902,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.0,0.0,2,138,0,1.386667,1.0,1.0,12.0,19.0,9.0,3.0,1.024,1.0,1.0,0.175
36488,ab633967-4f11-435b-a755-13015d242364,3,0.016594,35.0,0.041812,0.023787,0.018340,0.627314,49,10.892607,0.000000,0,0,0.011204,0,0.041317,0,262,169,59,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.0,0.0,2,706,7,1.386667,1.0,1.0,9.0,1.0,10.0,3.0,3.072,1.0,1.0,0.400
36489,ab9dda24-b688-4cb8-9ff8-27158438ff4d,0,0.158735,130.0,0.024390,0.008495,0.008319,0.623167,468,14.797651,0.000000,0,1,

In [15]:
X_npi = dev_step1.drop(['line_id', 'upgrade'], axis=1) # npi = no phone info
y_npi = dev_step1['upgrade']
X_eval_npi = eval_step1.drop(['line_id'], axis=1)

In [16]:
X_npi_train, X_npi_test, y_npi_train, y_npi_test = train_test_split(X_npi, y_npi, test_size=0.2)

In [17]:
X_wpi = dev_final.drop(['line_id', 'upgrade'], axis=1) # wpi = with phone info
y_wpi = dev_final['upgrade']
X_eval_wpi = eval_final.drop(['line_id'], axis=1)

In [18]:
X_wpi_train, X_wpi_test, y_wpi_train, y_wpi_test = train_test_split(X_wpi, y_wpi, test_size=0.2)

## Build model for samples without phone info

In [19]:
clf_npi = RandomForestClassifier(n_estimators=200, criterion='entropy')

In [20]:
clf_npi.fit(X_npi_train, y_npi_train)
y_npi_pred = clf_npi.predict(X_npi_test)
print(f1_score(y_npi_test, y_npi_pred))
print(classification_report(y_npi_test, y_npi_pred))

0.8759608665269044
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      8108
           1       0.88      0.88      0.88      2864

    accuracy                           0.94     10972
   macro avg       0.92      0.92      0.92     10972
weighted avg       0.94      0.94      0.94     10972



In [21]:
clf_npi.feature_importances_

array([1.89172557e-02, 6.49841081e-02, 2.74584497e-02, 1.15093769e-01,
       5.05733444e-02, 5.25581312e-02, 6.27394046e-02, 4.65816411e-02,
       3.53589480e-02, 1.15107854e-02, 3.09838441e-02, 1.95103079e-02,
       6.14485942e-02, 2.93118852e-02, 5.06831507e-02, 2.44803536e-02,
       3.99379991e-02, 6.74442581e-02, 5.17418229e-02, 1.52215428e-02,
       6.15870752e-03, 2.56876409e-03, 3.35578935e-03, 3.39867764e-02,
       1.79937729e-02, 1.27533417e-02, 2.89175125e-04, 0.00000000e+00,
       2.19320882e-03, 2.73988155e-05, 2.11745977e-06, 2.20153762e-05,
       4.80143620e-03, 3.83068326e-06, 9.58380028e-07, 2.40987586e-04,
       2.34951024e-03, 4.48707004e-03, 3.58850459e-05, 7.61259114e-04,
       6.01897823e-03, 1.41098404e-04, 1.56363974e-05, 1.97918172e-03,
       1.53876045e-04, 1.87810084e-03, 2.38662244e-06, 9.75713627e-06,
       2.30463430e-04, 2.54583127e-06, 1.40240538e-04, 0.00000000e+00,
       1.52395613e-05, 9.63422724e-06, 1.63852661e-02, 3.68992806e-06,
      

In [22]:
scores = cross_validate(clf_npi, X_npi, y_npi, cv=5, scoring='f1')
print(np.mean(scores['test_score']))

0.8687597219424458


In [23]:
# re-fit
clf_npi.fit(X_npi, y_npi)
y_eval_npi = clf_npi.predict(X_eval_npi)

In [24]:
# use a dicitionary to store lineid: prediction
result = {}
line_ids = eval_step1.loc[:,['line_id']].reset_index(drop=True)
for i in range(len(line_ids)):
    result[line_ids.loc[i][0]] = y_eval_npi[i]

In [25]:
len(result)

36491

## Build model for samples with phone info

In [26]:
clf_wpi = RandomForestClassifier(n_estimators=200, criterion='entropy')

In [27]:
clf_wpi.fit(X_wpi_train, y_wpi_train)
y_wpi_pred = clf_wpi.predict(X_wpi_test)
print(f1_score(y_wpi_test, y_wpi_pred))
print(classification_report(y_wpi_test, y_wpi_pred))

0.8585082069395387
              precision    recall  f1-score   support

           0       0.94      0.95      0.95      6307
           1       0.87      0.84      0.86      2446

    accuracy                           0.92      8753
   macro avg       0.91      0.90      0.90      8753
weighted avg       0.92      0.92      0.92      8753



In [28]:
clf_wpi.feature_importances_

array([1.34673234e-02, 4.97244189e-02, 1.81828612e-02, 1.12536077e-01,
       4.96401952e-02, 4.79354413e-02, 6.55384451e-02, 3.54232113e-02,
       2.69311305e-02, 8.80878304e-03, 2.56853857e-02, 1.48841579e-02,
       5.86667505e-02, 2.98982044e-02, 4.28495323e-02, 2.16562494e-02,
       3.38532017e-02, 5.56392567e-02, 4.29290377e-02, 1.89991402e-02,
       6.15809062e-03, 2.91763600e-03, 2.20378228e-03, 2.91329782e-02,
       1.48285802e-02, 9.78068422e-03, 2.04776827e-04, 9.68891402e-06,
       1.73179173e-03, 2.31585281e-05, 2.29840675e-06, 1.47147419e-05,
       3.77291399e-03, 2.59026526e-06, 1.75972114e-05, 1.73728271e-04,
       1.73534159e-03, 3.32388186e-03, 2.56870868e-05, 3.34807332e-04,
       4.77268010e-03, 1.04238037e-04, 8.29013888e-06, 1.60069293e-03,
       1.08074166e-04, 1.53618266e-03, 0.00000000e+00, 7.66390463e-06,
       2.34205936e-04, 0.00000000e+00, 9.30621615e-05, 0.00000000e+00,
       9.35368295e-06, 1.07850911e-06, 1.55412163e-02, 9.65262143e-06,
      

In [29]:
scores = cross_validate(clf_wpi, X_wpi, y_wpi, cv=5, scoring='f1')
print(np.mean(scores['test_score']))

0.8690620790077552


In [30]:
# re-fit
clf_wpi.fit(X_wpi, y_wpi)
y_eval_wpi = clf_wpi.predict(X_eval_wpi)

In [31]:
line_ids = eval_final.loc[:,['line_id']].reset_index(drop=True)
for i in range(len(line_ids)):
    result[line_ids.loc[i][0]] = y_eval_wpi[i] # line_ids with phone info will get their predications updated

In [32]:
len(result)

36491

## Randomly guess others

In [33]:
eval_upgrades = pd.read_csv(eval_folder + 'upgrades.csv')
whole_line_ids = eval_upgrades.loc[:, ['line_id']].values
for i in whole_line_ids:
    if i[0] not in result.keys():
        result[i[0]] = 0 # or guess other value

In [34]:
len(result)

37155

## Create submission

In [35]:
predictions = pd.DataFrame(columns=['line_id', 'upgrade'])
for k in result.keys():
    predictions = predictions.append([{'line_id': k, 'upgrade': result[k]}], ignore_index=True)

In [36]:
predictions.to_csv('2021-04-25.csv',header=True,index=None)

In [37]:
import boto3
s3 = boto3.resource('s3')

In [38]:
s3.meta.client.upload_file('2021-04-25.csv', 'tf-trachack-notebooks', teamname+'/jupyter/jovyan/submission/2021-04-25.csv')

In [39]:
predictions

,line_id,upgrade
0,584f0d75-5897-4e56-b475-0b3adf1aeb07,1
1,e879756a-daff-4f72-9206-d96009861ac1,1
2,cbb0795d-716a-44f1-bc82-f8f8cb5db002,0
3,e1e914fd-1e04-4148-9251-ed6a4dd6d3e7,0
4,1987025d-39be-4120-a09a-8d89c0e8d151,0
...,...,...
37150,37a28904-3e03-4c71-8964-d26b0e0e1395,0
37151,2adbd2d3-176e-4125-aca5-e39e31429121,0
37152,c7bcd176-41ea-4359-a805-cbb09614d734,0
37153,6bf24f82-349a-458c-b273-b2a245c40f25,0
